In [0]:
import tensorflow as tf
import numpy as np

In [0]:
def sigmoid_derivative(x):
    return x * tf.math.subtract(tf.ones(x.shape), x)

In [0]:
class NeuralNetwork:
    def __init__(self, x, y):
        self.input = tf.get_variable('input', shape = x.shape, initializer = tf.constant_initializer(x))
        self.output = tf.Variable(tf.zeros([len(y),], tf.int32))
        self.y = tf.get_variable('y', shape = [len(y),], initializer = tf.constant_initializer(y))
        self.y = tf.reshape(self.y, [len(y),1])
        self.weights1 = tf.random.normal(shape=[self.input.shape[1].value, 5])
        self.weights2 = tf.random.normal(shape=[5,1])
        self.bias1 = tf.random.normal(shape=[1,5])
        self.bias2 = tf.random.normal(shape=[1,1])
        self.data = tf.placeholder(tf.float32)
    
    def feedForward(self):
        self.layer1 = tf.sigmoid(tf.tensordot(self.input, self.weights1, 1) + self.bias1)
        self.output = tf.sigmoid(tf.tensordot(self.layer1, self.weights2, 1) + self.bias2)
    
    def backpropogation(self):
        input_t = tf.transpose(self.input)
        error = tf.math.subtract(self.y, self.output)
        weight2_t = tf.transpose(self.weights2)

        d_weights_1 = tf.tensordot( input_t,
                                   (tf.tensordot(2*(error) * sigmoid_derivative(self.output),
                                                 weight2_t,
                                                 1) * sigmoid_derivative(self.layer1)),
                                    1)
        
        d_weights_2 = tf.tensordot(tf.transpose(self.layer1), (2*(tf.math.subtract(self.y, self.output)) * sigmoid_derivative(self.output)), 1)
        
        d_bias_1 = tf.math.reduce_mean((tf.tensordot(2*(error) * sigmoid_derivative(self.output),
                                                 weight2_t,
                                                 1) * sigmoid_derivative(self.layer1)),
                                        0)
        d_bias_2 = tf.math.reduce_mean(2*(tf.math.subtract(self.y, self.output)) * sigmoid_derivative(self.output), 0)
        
        self.weights1 += d_weights_1
        self.weights2 += d_weights_2
        self.bias1 += d_bias_1
        self.bias2 += d_bias_2
    
    def trainModel(self, epoch):
        for _ in range(1, epoch):
            self.feedForward()
            self.backpropogation()
            
    def predict(self, data):
        return sess.run(self.predictResult(), feed_dict={
            self.data:data
        })
        
    def predictResult(self):
        layer1 = tf.sigmoid(tf.tensordot(self.data, self.weights1, 1) + self.bias1)
        result = tf.sigmoid(tf.tensordot(layer1, self.weights2, 1) + self.bias2)
        return result

In [0]:
train_data = np.matrix([[0, 0, 0],
                        [0, 0, 1],
                        [0, 1, 0],
                        [0, 1, 1],
                        [1, 0, 0],
                        [1, 0, 1],
                        [1, 1, 0],
                        [1, 1, 1]])
train_target = np.array([0, 1, 1, 0, 1, 0, 0, 1])

In [0]:
tf.reset_default_graph()

In [0]:
model = NeuralNetwork(train_data, train_target)

In [0]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [0]:
model.trainModel(1500)
result = sess.run(model.output)

In [151]:
print(result)

[[0.02017964]
 [0.97695196]
 [0.9790156 ]
 [0.01714825]
 [0.97482   ]
 [0.01702505]
 [0.01855114]
 [0.9917897 ]]


In [152]:
test_data = np.matrix([[0, 1, 0],
                       [1, 0, 0],
                       [1, 1, 0]])
test_result = model.predict(test_data)
print(test_result)

[[0.98801327]
 [0.9686924 ]
 [0.02868488]]
